In [1]:
from datasets import load_dataset
from transformers import RobertaTokenizer
from transformers import TrainingArguments, Trainer
from transformers import RobertaForSequenceClassification
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

In [2]:
dataset_loc = "dair-ai/emotion"
dataset_cache_dir = "../data/raw"
model_loc = "distilroberta-base"

In [3]:
dataset = load_dataset(dataset_loc, cache_dir = dataset_cache_dir)

In [4]:
tokenizer = RobertaTokenizer.from_pretrained(model_loc)

In [5]:
model = RobertaForSequenceClassification.from_pretrained(model_loc, num_labels = 6)

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at distilroberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [16]:
training_args = TrainingArguments(
    output_dir = "./results",
    eval_strategy = "epoch",
    save_strategy = "epoch",
    learning_rate = 2e-5,
    per_device_train_batch_size = 16,
    per_device_eval_batch_size = 2,
    num_train_epochs = 1,
    weight_decay = 0.01,
    logging_steps = 5,
    logging_strategy ="steps",
    logging_first_step =True,
    report_to = "tensorboard",       
    disable_tqdm = False, 
    bf16 = True, 
    logging_dir = "logs",
    no_cuda = True
)

In [17]:
def tokenize_function(example):
    return tokenizer(example['text'], truncation = True, padding = 'max_length', max_length = 128)

tokenized_datasets = dataset.map(tokenize_function, batched = True)

In [18]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = logits.argmax(axis=-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='weighted')
    acc = accuracy_score(labels, preds)
    return {"accuracy": acc, "f1": f1, "precision": precision, "recall": recall}

In [26]:
trainer = Trainer(
    model = model,
    args = training_args,
    train_dataset = tokenized_datasets["train"].select(range(4)),
    eval_dataset = tokenized_datasets["test"].select(range(4)),
    compute_metrics = compute_metrics,
)

In [27]:
trainer.train()

/home/machine-x/miniconda3/envs/mlops/lib/python3.13/site-packages/transformers/trainer.py:3700: FutureWarning: `torch.cpu.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cpu', args...)` instead.
  ctx_manager = torch.cpu.amp.autocast(cache_enabled=cache_enabled, dtype=self.amp_dtype)


Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,1.867100,1.857628,0.250000,0.100000,0.062500,0.250000
2,1.867100,1.799595,0.250000,0.100000,0.062500,0.250000


/home/machine-x/miniconda3/envs/mlops/lib/python3.13/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/machine-x/miniconda3/envs/mlops/lib/python3.13/site-packages/transformers/trainer.py:3700: FutureWarning: `torch.cpu.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cpu', args...)` instead.
  ctx_manager = torch.cpu.amp.autocast(cache_enabled=cache_enabled, dtype=self.amp_dtype)
/home/machine-x/miniconda3/envs/mlops/lib/python3.13/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/hom

KeyboardInterrupt: 

In [ ]:
trainer.evaluate()